In [78]:
"""
Создание частотных таблиц для каждой из колонок таблицы

"""

'\nСоздание частотных таблиц для каждой из колонок таблицы\n\n'

In [79]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time

In [80]:
name_file_data_groupby = 'data/Эксперты ВСР.xlsx'
path_to_end_folder_groupby = 'data'

df = pd.read_excel(name_file_data_groupby)

print(f'Колонки в таблице {df.columns}')
# Добавляем столбец для облегчения подсчета по категориям
df['Для подсчета'] = 1
# Создаем шрифт которым будем выделять названия таблиц
font_name_table = Font(name='Arial Black', size=15, italic=True)
# Создаем файл excel
wb = openpyxl.Workbook()

# Делаем названия колонок строковыми
df.columns = list(map(str,list(df.columns)))

# Создаем листы
for idx,name_column in enumerate(df.columns):
    # Делаем короткое название не более 15 символов
    wb.create_sheet(title=name_column[:15],index=idx)

Колонки в таблице Index(['ФИО', '5', 'Организация, город   Отметка о пролонгации свидетельства',
       'Регион', '№ свидетельства', 'Компетенция', 'Дата выдачи',
       'Срок действия (лет)', 'Телефон', 'Имейл'],
      dtype='object')


In [81]:

for idx,name_column in enumerate(df.columns):
    group_df = df.groupby([name_column]).agg({'Для подсчета': 'sum'})
    group_df.columns = ['Количество']
    

        
    # Сортируем по убыванию
    group_df.sort_values(by=['Количество'],inplace=True,ascending=False)
    

    for r in dataframe_to_rows(group_df, index=True, header=True):
        if len(r) != 1:
            wb[name_column[:15]].append(r)
    wb[name_column[:15]].column_dimensions['A'].width = 50
    


In [82]:

# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Удаляем листы 
del wb['Sheet']
del wb['Для подсчета']
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_groupby}/Подсчет частоты значений для всех колонок таблицы от {current_time}.xlsx')